<a href="https://colab.research.google.com/github/tomtomh512/Handwritten-Arithmetic-Calculator-v2/blob/main/arithmetic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import pair_confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
od.download("https://www.kaggle.com/datasets/sagyamthapa/handwritten-math-symbols/data")

Dataset URL: https://www.kaggle.com/datasets/sagyamthapa/handwritten-math-symbols


100%|██████████| 39.4M/39.4M [00:00<00:00, 95.8MB/s]


Remove variables

In [ ]:
!rm -r /content/handwritten-math-symbols/dataset/x
!rm -r /content/handwritten-math-symbols/dataset/y
!rm -r /content/handwritten-math-symbols/dataset/z

Transform composition

In [ ]:
transform = transforms.Compose([
  transforms.Resize((128, 128)),          # Resize to a 128x128
  transforms.Grayscale(),                 # Convert to grayscale
  transforms.ToTensor(),                  # Convert to tensor
  transforms.Normalize([0.5], [0.5])      # Normalize
])

 Train and test data

In [ ]:
train_data = datasets.ImageFolder(root='/content/handwritten-math-symbols/dataset', transform=transform)
test_data = datasets.ImageFolder(root='/content/handwritten-math-symbols/dataset', transform=transform)
train_loader = DataLoader(train_data, batch_size = 30, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 30, shuffle = False)

Convolutional Network class

In [ ]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)       # input, output, kernel, stride
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.conv3 = nn.Conv2d(64, 128, 3, 1)
    self.conv4 = nn.Conv2d(128, 256, 3, 1)

    self.fc1 = nn.Linear(256 * 6 * 6, 512)    # 256 * 6 * 6 -> 512
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 128)
    self.fc4 = nn.Linear(128, 16)

  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X, 2, 2)
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2, 2)
    X = F.relu(self.conv3(X))
    X = F.max_pool2d(X, 2, 2)
    X = F.relu(self.conv4(X))
    X = F.max_pool2d(X, 2, 2)

    X = X.view(-1, 256 * 6 * 6)

    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = F.relu(self.fc3(X))
    X = self.fc4(X)

    return F.log_softmax(X, dim = 1)

Create CNN object

In [ ]:
model = ConvolutionalNetwork()

Using Cross Entropy and Adam Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

Training model

In [ ]:
epochs = 5

model.train()
for epoch in range(epochs):
  running_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    images, labels = data

    optimizer.zero_grad()

    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # images.size(0) = number of samples in batch, total loss for batch
    running_loss += loss.item() * images.size(0)

  epoch_loss = running_loss / len(train_loader.dataset)
  print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.3f}')

print('Finished Training')

Epoch 1/5, Loss: 1.181
Epoch 2/5, Loss: 0.295
Epoch 3/5, Loss: 0.150
Epoch 4/5, Loss: 0.100
Epoch 5/5, Loss: 0.080
Finished Training


Calculate Accuracy

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for i, data in enumerate(test_loader, 0):
    images, labels = data

    outputs = model(images)
    predicted = outputs.argmax(dim=1)
    total += labels.size(0)

    # Count number of matches
    correct += (predicted == labels).sum().item()

accuracy = correct / total * 100
print(f'Accuracy: {accuracy:.3f}%')

Accuracy: 99.356%


Save model state

In [ ]:
torch.save(model.state_dict(), 'math_model_0.pt')

Testing CNN

In [ ]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert('L')     # 'L' = Single channel image, grayscale
    transform = transforms.Compose([                # Match transform composition
        transforms.Resize((128, 128)),
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    image = transform(image)
    return image

image_path = '/content/handwritten-math-symbols/dataset/5/1102.jpg'
processed_image = preprocess_image(image_path)

In [ ]:
processed_image = processed_image.unsqueeze(0)  # Add batch dimension
model.eval()
with torch.no_grad():
    new_prediction = model(processed_image)

print(new_prediction.argmax())

tensor(5)
